In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns

/home/dell/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
etth1data = pd.read_csv('dataset/ETT-small/ETTh1.csv')
etth1data = etth1data.drop('date', axis=1)
target = etth1data.pop('OT')
etth1data.insert(0, 'OT', target)
data1 = etth1data
# pcmci_result = np.array([8, -1, 10, -1, 11,  4, 16, 20, 14, -1, 19, 13, -1,  9, 13,  7, 10,
#                             -1, 17, 17,  3, 16,  0, 16, 13])
pcmci_result = np.array([1, 1, 3, 1, 3, 2, 1]).astype(int)

In [4]:
metrodata = pd.read_csv('dataset/Metro_Interstate_Traffic_Volume.csv').drop(['weather_description', 'date_time'], axis=1)
target = metrodata.pop('traffic_volume')
metrodata.insert(0, 'OT', target)
metrodata.loc[metrodata['holiday'] == 'None', 'holiday'] = 0
metrodata.loc[metrodata['holiday'] != 0, 'holiday'] = 1
metrodata['holiday'] = metrodata['holiday'].astype('int64')
metrodata['weather_main'] = metrodata['weather_main'].map({'Clouds':0. , 'Clear':1., 'Rain':2., 'Drizzle':3., 'Mist':4., 'Haze':5., 'Fog':6.,
                                                            'Thunderstorm':7., 'Snow':8., 'Squall':9., 'Smoke':10.})
data1 = metrodata
pcmci_result = np.array([1, 1, 1, 1, 1, 1, 1]).astype(int)

In [16]:
exchangedata = pd.read_csv('dataset/exchange_rate/exchange_rate.csv')
exchangedata = exchangedata.drop('date', axis=1)
target = exchangedata.pop('OT')
exchangedata.insert(0, 'OT', target)
data1 = exchangedata
pcmci_result = np.array([1, -1, 3, 5, 3, -1, 1, 1]).astype(int)

In [3]:
pmdata = pd.read_csv('dataset/beijing_air.csv')
pmdata['date'] = pd.to_datetime(pmdata['date'])
pmdata['year'] = pmdata['date'].apply(lambda x:x.year)
pmdata['month'] = pmdata['date'].apply(lambda x:x.month)
pmdata['day'] = pmdata['date'].apply(lambda x:x.day)
pmdata['hour'] = pmdata['date'].apply(lambda x:x.hour)
pmdata = pmdata.drop(['date', 'wnd_dir'], axis=1)
pmdata.dtypes

data1 = pmdata
pcmci_result = np.array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]).astype(int)

In [4]:
sorted_data = data1.iloc[:, pcmci_result > -1]
sorted_arrows = pcmci_result[pcmci_result > -1].astype(int)

tau_max = 20
ar_range = 10

lagged_data = pd.DataFrame(sorted_data.iloc[tau_max:, 0])
for i in range(1, ar_range):
    lagged_data['OT' + '_' + str(i)] = np.array(sorted_data.iloc[tau_max-i:-i, 0])
for i in range(sorted_data.shape[1]):
    if sorted_arrows[i] == 0:
        lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max:, i])
        continue
    lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max - sorted_arrows[i]:-sorted_arrows[i], i])

minmaxgap =  lagged_data.iloc[:, 0].max() - lagged_data.iloc[:, 0].min()
lagged_data = (lagged_data - lagged_data.min()) / ( lagged_data.max() - lagged_data.min())

data2 = lagged_data.copy()
data = np.array(data2)
data.shape

train_proportion = 0.6
valid_proportion = 0.2
test_proportion = 0.2

train_size = int(data.shape[0] * train_proportion)
valid_size = int(data.shape[0] * valid_proportion)

train_data = data[:train_size]
valid_data = data[train_size:train_size+valid_size]
test_data = data[train_size+valid_size:]

batch_size = 20
window_size = 400

z_size = data.shape[1]


Z = np.load('results/new_exchange_rate_z.npy')

train_Z = Z[:train_size]
valid_Z = Z[train_size:train_size+valid_size]
test_Z = Z[train_size+valid_size:]

###
valid_Z = torch.autograd.Variable(torch.FloatTensor(valid_Z)).cuda()
###

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_1199165/2110840739.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [16:05<00:00, 51.77it/s]


0.0005784976986138886


In [6]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_1199165/1802000487.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)


100%|██████████| 50000/50000 [16:09<00:00, 51.56it/s]


0.0005867390404610108


In [7]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_1199165/3099682845.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [21:31<00:00, 38.71it/s]

0.0005699555803152768


In [8]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        
        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipykernel_1199165/105458899.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [22:08<00:00, 37.64it/s]


0.0005873271280338906


In [11]:
etth2data = pd.read_csv('dataset/ETT-small/ETTh2.csv')
etth2data = etth2data.drop('date', axis=1)
target = etth2data.pop('OT')
etth2data.insert(0, 'OT', target)
data1 = etth2data
pcmci_result = np.array([1, 1, 1, 1, 1, 1, 1]).astype(int)

In [28]:
pmdata = pd.read_csv('dataset/beijing_air.csv')
pmdata['date'] = pd.to_datetime(pmdata['date'])
pmdata['year'] = pmdata['date'].apply(lambda x:x.year)
pmdata['month'] = pmdata['date'].apply(lambda x:x.month)
pmdata['day'] = pmdata['date'].apply(lambda x:x.day)
pmdata['hour'] = pmdata['date'].apply(lambda x:x.hour)
pmdata = pmdata.drop(['date', 'wnd_dir'], axis=1)
pmdata.dtypes

data1 = pmdata
pcmci_result = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]).astype(int)

In [29]:
sorted_data = data1.iloc[:, pcmci_result > -1]
sorted_arrows = pcmci_result[pcmci_result > -1].astype(int)

tau_max = 20
ar_range = 10

lagged_data = pd.DataFrame(sorted_data.iloc[tau_max:, 0])
for i in range(1, ar_range):
    lagged_data['Target' + '_' + str(i)] = np.array(sorted_data.iloc[tau_max-i:-i, 0])
for i in range(sorted_data.shape[1]):
    if sorted_arrows[i] == 0:
        lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max:, i])
        continue
    lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max - sorted_arrows[i]:-sorted_arrows[i], i])

minmaxgap =  lagged_data.iloc[:, 0].max() - lagged_data.iloc[:, 0].min()
lagged_data = (lagged_data - lagged_data.min()) / ( lagged_data.max() - lagged_data.min())

data2 = lagged_data.copy()
data = np.array(data2)
data.shape

train_proportion = 0.6
valid_proportion = 0.2
test_proportion = 0.2

train_size = int(data.shape[0] * train_proportion)
valid_size = int(data.shape[0] * valid_proportion)

train_data = data[:train_size]
valid_data = data[train_size:train_size+valid_size]
test_data = data[train_size+valid_size:]

batch_size = 20
window_size = 400

z_size = data.shape[1]


Z = np.load('results/pm_z.npy')

train_Z = Z[:train_size]
valid_Z = Z[train_size:train_size+valid_size]
test_Z = Z[train_size+valid_size:]

###
valid_Z = torch.autograd.Variable(torch.FloatTensor(valid_Z)).cuda()
###

In [30]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]
    
        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_967354/2681342656.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [15:58<00:00, 52.19it/s]

0.0005847809841388942


In [31]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_967354/4262776442.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [15:55<00:00, 52.34it/s]


0.000587423369750537


In [32]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_967354/3294305005.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [21:17<00:00, 39.13it/s]

0.0005830097096143605


In [33]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_967354/1802000487.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [23:06<00:00, 36.05it/s]


0.0005857808317175381


In [25]:
ettm1data = pd.read_csv('dataset/ETT-small/ETTm1.csv')
ettm1data = ettm1data.drop('date', axis=1)
target = ettm1data.pop('OT')
ettm1data.insert(0, 'OT', target)
data1 = ettm1data
pcmci_result = np.array([1, 1, 1, 1, 1, 1, 1]).astype(int)

In [26]:
sorted_data = data1.iloc[:, pcmci_result > -1]
sorted_arrows = pcmci_result[pcmci_result > -1].astype(int)

tau_max = 20
ar_range = 10

lagged_data = pd.DataFrame(sorted_data.iloc[tau_max:, 0])
for i in range(1, ar_range):
    lagged_data['Target' + '_' + str(i)] = np.array(sorted_data.iloc[tau_max-i:-i, 0])
for i in range(sorted_data.shape[1]):
    if sorted_arrows[i] == 0:
        lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max:, i])
        continue
    lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max - sorted_arrows[i]:-sorted_arrows[i], i])

minmaxgap =  lagged_data.iloc[:, 0].max() - lagged_data.iloc[:, 0].min()
lagged_data = (lagged_data - lagged_data.min()) / ( lagged_data.max() - lagged_data.min())

data2 = lagged_data.copy()
data = np.array(data2)
data.shape

train_proportion = 0.6
valid_proportion = 0.2
test_proportion = 0.2

train_size = int(data.shape[0] * train_proportion)
valid_size = int(data.shape[0] * valid_proportion)

train_data = data[:train_size]
valid_data = data[train_size:train_size+valid_size]
test_data = data[train_size+valid_size:]

batch_size = 20
window_size = 400

z_size = data.shape[1]


Z = np.load('results/ettm1_z.npy')

train_Z = Z[:train_size]
valid_Z = Z[train_size:train_size+valid_size]
test_Z = Z[train_size+valid_size:]

###
valid_Z = torch.autograd.Variable(torch.FloatTensor(valid_Z)).cuda()
###

In [27]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipykernel_5021/2110840739.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [23:28<00:00, 35.50it/s]


4.2374494963350835e-05


In [28]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_5021/4262776442.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
  0%|          | 20/50000 [00:00<26:00, 32.02it/s]

100%|██████████| 50000/50000 [23:32<00:00, 35.39it/s]


4.350068194598595e-05


In [30]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipykernel_5021/3294305005.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [34:37<00:00, 24.06it/s]


5.121707533779109e-05


In [31]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_5021/1802000487.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
  0%|          | 15/50000 [00:00<33:39, 24.75it/s]

100%|██████████| 50000/50000 [33:44<00:00, 24.69it/s]

7.565377045861268e-05


In [3]:
ettm2data = pd.read_csv('dataset/ETT-small/ETTm2.csv')
ettm2data = ettm2data.drop('date', axis=1)
target = ettm2data.pop('OT')
ettm2data.insert(0, 'OT', target)
data1 = ettm2data
pcmci_result = np.array([1, 1, 1, 1, 1, 1, 1]).astype(int)

In [4]:
sorted_data = data1.iloc[:, pcmci_result > -1]
sorted_arrows = pcmci_result[pcmci_result > -1].astype(int)

tau_max = 20
ar_range = 10

lagged_data = pd.DataFrame(sorted_data.iloc[tau_max:, 0])
for i in range(1, ar_range):
    lagged_data['Target' + '_' + str(i)] = np.array(sorted_data.iloc[tau_max-i:-i, 0])
for i in range(sorted_data.shape[1]):
    if sorted_arrows[i] == 0:
        lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max:, i])
        continue
    lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max - sorted_arrows[i]:-sorted_arrows[i], i])

minmaxgap =  lagged_data.iloc[:, 0].max() - lagged_data.iloc[:, 0].min()
lagged_data = (lagged_data - lagged_data.min()) / ( lagged_data.max() - lagged_data.min())

data2 = lagged_data.copy()
data = np.array(data2)
data.shape

train_proportion = 0.6
valid_proportion = 0.2
test_proportion = 0.2

train_size = int(data.shape[0] * train_proportion)
valid_size = int(data.shape[0] * valid_proportion)

train_data = data[:train_size]
valid_data = data[train_size:train_size+valid_size]
test_data = data[train_size+valid_size:]

batch_size = 20
window_size = 400

z_size = data.shape[1]


Z = np.load('results/ettm2_z.npy')

train_Z = Z[:train_size]
valid_Z = Z[train_size:train_size+valid_size]
test_Z = Z[train_size+valid_size:]

###
valid_Z = torch.autograd.Variable(torch.FloatTensor(valid_Z)).cuda()
###

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_586517/2110840739.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
100%|██████████| 50000/50000 [26:31<00:00, 31.43it/s]


2.0292250798249835e-05


In [6]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_586517/4262776442.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
  0%|          | 3/50000 [00:00<28:47, 28.94it/s]

100%|██████████| 50000/50000 [23:03<00:00, 36.13it/s]


3.07195284975185e-05


In [7]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_586517/3294305005.py:55: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
  0%|          | 3/50000 [00:00<33:07, 25.16it/s]

100%|██████████| 50000/50000 [32:50<00:00, 25.37it/s]


3.7681610767417165e-05


In [8]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_586517/1802000487.py:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)


100%|██████████| 50000/50000 [28:22<00:00, 29.37it/s]


8.458490606526428e-05


In [3]:
exchangedata = pd.read_csv('dataset/exchange_rate/exchange_rate.csv')
exchangedata = exchangedata.drop('date', axis=1)
target = exchangedata.pop('OT')
exchangedata.insert(0, 'OT', target)
data1 = exchangedata
pcmci_result = np.array([1, -1, 3, 5, 3, -1, 1, 1]).astype(int)

In [4]:
sorted_data = data1.iloc[:, pcmci_result > -1]
sorted_arrows = pcmci_result[pcmci_result > -1].astype(int)

tau_max = 20
ar_range = 10

lagged_data = pd.DataFrame(sorted_data.iloc[tau_max:, 0])
for i in range(1, ar_range):
    lagged_data['Target' + '_' + str(i)] = np.array(sorted_data.iloc[tau_max-i:-i, 0])
for i in range(sorted_data.shape[1]):
    if sorted_arrows[i] == 0:
        lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max:, i])
        continue
    lagged_data[sorted_data.columns[i] + '_' + str(sorted_arrows[i])] = np.array(sorted_data.iloc[tau_max - sorted_arrows[i]:-sorted_arrows[i], i])

minmaxgap =  lagged_data.iloc[:, 0].max() - lagged_data.iloc[:, 0].min()
lagged_data = (lagged_data - lagged_data.min()) / ( lagged_data.max() - lagged_data.min())

data2 = lagged_data.copy()
data = np.array(data2)
data.shape

train_proportion = 0.6
valid_proportion = 0.2
test_proportion = 0.2

train_size = int(data.shape[0] * train_proportion)
valid_size = int(data.shape[0] * valid_proportion)

train_data = data[:train_size]
valid_data = data[train_size:train_size+valid_size]
test_data = data[train_size+valid_size:]

batch_size = 20
window_size = 50

z_size = data.shape[1]


Z = np.load('results/exchange_rate_z.npy')

train_Z = Z[:train_size]
valid_Z = Z[train_size:train_size+valid_size]
test_Z = Z[train_size+valid_size:]

###
valid_Z = torch.autograd.Variable(torch.FloatTensor(valid_Z)).cuda()
###


In [5]:
train_size

4540

In [6]:
valid_Z.shape

torch.Size([1513, 15])

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()


    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = 256, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = 256, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


  0%|          | 0/50000 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 31, got 30

In [ ]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()
min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)



NameError: name 'RNN' is not defined

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size = 2, hidden_size = 32, num_layers = 1, output_size = 2):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,     # rnn hidden unit
            num_layers = num_layers,       # number of rnn layer
            batch_first = True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        # print(r_out.shape)
        # outs = []    # save all predictions
        # for time_step in range(r_out.size(1)):    # calculate output for each time step
        #     outs.append(self.out(r_out[:, time_step, :]))

        # return torch.stack(outs, dim=1), h_state

        outs = self.out(r_out)
        return outs, h_state

def train_reg(data, Z, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    # X_lagged = np.concatenate((np.ones(shape=(1,1), dtype=np.float)*data[0,0], np.expand_dims((data[0:-1:, 0]), 0)), 1)
    # Y = np.concatenate((X_lagged.T, data[:, 1:]), 1)
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]
        Z_ = Z[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(torch.cat((Y_, Z_), axis = 1), h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(torch.cat((valid_Y, valid_Z), axis = 1), h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()


def test(data, Z, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()
    Z = torch.autograd.Variable(torch.FloatTensor(Z)).cuda()
    
    h_state = None
    X_pred, h_state = regr_net(torch.cat((Y, Z), axis = 1), h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()


min_loss = 1000
for i in range(10):
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_optim = optim.Adam(regr_net.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred = train_reg(train_data, train_Z, regr_net, regr_net_optim, iter_num = 50000)
    # loss_array, train_X_pred = train_reg(valid_data, valid_Z, regr_net, regr_net_optim, iter_num = 50000)

    regr_trained = regr_net
    regr_net = RNN(input_size = data.shape[1]-1+z_size, hidden_size = z_size * 4, num_layers = 2, output_size = 1).cuda()
    regr_net.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred = test(test_data, test_Z, regr_net)
    MSEloss = np.mean(np.square(test_data[10:,0] - X_pred[10:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)


NameError: name 'nn' is not defined

In [ ]:
def test_without_Z(data, regr_net):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    h_state = None
    X_pred, h_state = regr_net(Y, h_state)
    mseloss = torch.nn.MSELoss(reduction = 'sum')
    loss = mseloss(X_pred, X)
    return loss.detach(), X_pred.detach().cpu().numpy()

def train_reg_withoutZ(data, regr_net, regr_net_optim, iter_num = 5000):
    X = np.expand_dims(data[:, 0], 1)
    Y = data[:, 1:]
    X = torch.autograd.Variable(torch.FloatTensor(X)).cuda()
    Y = torch.autograd.Variable(torch.FloatTensor(Y)).cuda()

    ###
    valid_X = np.expand_dims(valid_data[:, 0], 1)
    valid_Y = valid_data[:, 1:]
    valid_X = torch.autograd.Variable(torch.FloatTensor(valid_X)).cuda()
    valid_Y = torch.autograd.Variable(torch.FloatTensor(valid_Y)).cuda()
    ###

    best_result = 1000
    loss_array = []
    for _ in tqdm(range(iter_num)):
        pivot = np.random.randint(X.shape[0]-window_size)
        X_ = X[pivot:pivot+window_size]
        Y_ = Y[pivot:pivot+window_size]

        h_state = None
        X_pred, h_state = regr_net(Y_, h_state)
        mseloss = torch.nn.MSELoss(reduction = 'mean')
        loss = mseloss(X_pred, X_)
        regr_net_optim.zero_grad()
        autograd.backward(loss)
        nn.utils.clip_grad_norm(regr_net.parameters(), max_norm=0.001, norm_type=2)
        regr_net_optim.step()
        loss_array.append(loss.detach().cpu().numpy())

        ###
        h_state = None
        X_valid_pred, h_state = regr_net(valid_Y, h_state)
        loss = mseloss(valid_X, X_valid_pred).detach().cpu().numpy()
        ###

        if loss < best_result:
            best_result = loss
            torch.save(regr_net.state_dict(), 'regr_net.pt')
    
    return loss_array, X_pred.detach().cpu().numpy()

min_loss = 1000
for i in range(10):
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z_optim = optim.Adam(regr_net_without_Z.parameters(), lr = 1e-3, weight_decay = 1e-5)
    loss_array, train_X_pred_without_Z = train_reg_withoutZ(train_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)
    # loss_array, train_X_pred_without_Z = train_reg_withoutZ(valid_data, regr_net_without_Z, regr_net_without_Z_optim, iter_num = 50000)

    regr_without_Z_trained = regr_net_without_Z
    regr_net_without_Z = RNN(input_size = data.shape[1]-1, hidden_size = data.shape[1] * 4, num_layers = 2, output_size = 1).cuda()
    regr_net_without_Z.load_state_dict(torch.load('regr_net.pt'))

    test_loss, X_pred_without_Z = test_without_Z(test_data, regr_net_without_Z)
    MSEloss = np.mean(np.square(test_data[:,0] - X_pred_without_Z[:].T))# * minmaxgap
    if MSEloss < min_loss:
        min_loss = MSEloss

print(min_loss)

NameError: name 'RNN' is not defined